## Selección de características para mejorar modelos predictivos - BreastCancer

En este cuadernillo se realiza el tratamiento de datos del csv BreastCancer, así como el uso de los algoritmos de búsqueda secuencial hacia atrás y hacia atrás mixto. Además del uso de algoritmos de entrenamiento como los son `DecisionTreeClassifier` y `kNN` para calcular diferentes métricas y evaluar la calidad de las soluciones.

### Tratamiento de datos

Se aplicaron los siguientes procesos de preprocesamiento de datos:

    •Normalización de variables predictoras: Se normalizaron las variables continuas utilizando el escalador MinMaxScaler para asegurar que todas las características estén en la misma escala.

    •Codificación numérica de atributos discretos: En este archivo CSV no hay ningún atributo discreto por lo que no hubo que codificar.

### Primer experimento para evaluar la capacidad predictiva del conjunto de variables completo

    Para evaluar la calidad de las soluciones que se obtienen con los algoritmos de búsqueda de variables, se realizó un primer experimento en el que se entrenó un modelo de clasificación utilizando todas las variables predictoras disponibles. Se utilizó un árbol de decisión de clasificación y el algoritmo kNN. Además se dividió en conjunto de datos en entrenamiento y prueba (80% y 20% respectivamente) y se uso una búsqueda en rejilla para optimizar los hiperparámetros y así mejorar el rendimiento.

### Algoritmos de búsqueda implementados

    •Búsqueda secuencial hacia atrás (backward_sequential_search): Este algoritmo busca encontrar el mejor subconjunto de variables predictoras eliminando iterativamente la variable que más afecta el rendimiento del modelo.

    •Búsqueda secuencial hacia atrás mixta (backward_sequential_search_mixto): Similar al anterior, pero también considera añadir variables si se mejora el rendimiento del modelo.

### Algoritmos de entrenamiento

    •Árboles de decisión de clasificación (DecisionTreeClassifier): Un algoritmo de aprendizaje supervisado utilizado para clasificación.
    
    •kNN (k-nearest neighbors): Otro algoritmo usado para clasificación que nos aporta más información.

## Tratamiento de datos

In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder
from sklearn.preprocessing import MinMaxScaler


model = RandomForestClassifier()

breast_cancer = pd.read_csv('../data/BreastCancer.csv')


atributos_continuos = ['mean radius' ,'mean texture' ,'mean perimeter' ,'mean area' ,'mean smoothness' ,'mean compactness' ,'mean concavity' ,
                       'mean concave points' ,'mean symmetry' ,'mean fractal dimension' ,'radius error' ,'texture error' ,'perimeter error' ,
                       'area error' ,'smoothness error' ,'compactness error' ,'concavity error' ,'concave points error','symmetry error','fractal dimension error',
                       'worst radius' ,'worst texture' ,'worst perimeter','worst area' ,'worst smoothness','worst compactness' ,'worst concavity' ,'worst concave points','worst symmetry','worst fractal dimension']
atributos = breast_cancer.loc[:, atributos_continuos]

objetivo = breast_cancer['diagnosis']
 

normalizador = MinMaxScaler(
    # Cada atributo se normaliza al intervalo [0, 1]
    feature_range=(0, 1)
)

# Como nos interesa conservar los atributos originales, realizamos la
# normalización sobre una copia del DataFrame de atributos
atributos_normalizados = atributos.copy()
atributos_normalizados[:] = normalizador.fit_transform(atributos_normalizados)


breast_cancer = atributos_normalizados.copy()
breast_cancer['Diagnosis'] = objetivo
breast_cancer.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,Diagnosis
0,0.521037,0.022658,0.545989,0.363733,0.593753,0.792037,0.703140,0.731113,0.686364,0.605518,...,0.141525,0.668310,0.450698,0.601136,0.619292,0.568610,0.912027,0.598462,0.418864,0
1,0.643144,0.272574,0.615783,0.501591,0.289880,0.181768,0.203608,0.348757,0.379798,0.141323,...,0.303571,0.539818,0.435214,0.347553,0.154563,0.192971,0.639175,0.233590,0.222878,0
2,0.601496,0.390260,0.595743,0.449417,0.514309,0.431017,0.462512,0.635686,0.509596,0.211247,...,0.360075,0.508442,0.374508,0.483590,0.385375,0.359744,0.835052,0.403706,0.213433,0
3,0.210090,0.360839,0.233501,0.102906,0.811321,0.811361,0.565604,0.522863,0.776263,1.000000,...,0.385928,0.241347,0.094008,0.915472,0.814012,0.548642,0.884880,1.000000,0.773711,0
4,0.629893,0.156578,0.630986,0.489290,0.430351,0.347893,0.463918,0.518390,0.378283,0.186816,...,0.123934,0.506948,0.341575,0.437364,0.172415,0.319489,0.558419,0.157500,0.142595,0


## Evaluación del modelo

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate


(atributos_entrenamiento, atributos_prueba,
 objetivo_entrenamiento, objetivo_prueba) = train_test_split(
        atributos, objetivo,
        # Tamaño del conjunto de prueba (20 % en este caso)
        test_size=.2,
        stratify=objetivo)

### Árbol de decisión de clasificación(DecisionTreeClassifier)

In [3]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

clasificador_CART = DecisionTreeClassifier(random_state=42)
rejilla_de_hiperparámetros = {
    # Máxima profundidad del árbol: 3, 4, 5, 6, 7, 8, 9, 10
    'max_depth': range(3, 11),
    # Mínimo número de ejemplos para poder particionar: 5, 10, 15
    'min_samples_split': range(5, 20, 5)
}
clasificador_CART.fit(atributos_entrenamiento, objetivo_entrenamiento)

búsqueda_en_rejilla = GridSearchCV(clasificador_CART,
                                   rejilla_de_hiperparámetros,
                                   scoring='balanced_accuracy',
                                   cv=10, 
                                   n_jobs=-1)  

búsqueda_en_rejilla.fit(atributos_entrenamiento, objetivo_entrenamiento)

# Obtener los mejores parámetros de la búsqueda en rejilla
mejores_parametros = búsqueda_en_rejilla.best_params_

# Guardar el valor de max_depth en una variable
max_depth_mejor = mejores_parametros.get('max_depth')
min_samples_split_mejor = mejores_parametros.get('min_samples_split')
mejor_score = búsqueda_en_rejilla.best_score_

print("Mejor valor de max_depth:", max_depth_mejor)
print("Mejor valor de min_samples_split:", min_samples_split_mejor)
print("Mejor score:", mejor_score)


Mejor valor de max_depth: 6
Mejor valor de min_samples_split: 5
Mejor score: 0.9207657200811358


In [4]:
resultados_validación_cruzada = cross_validate(clasificador_CART,
                                               atributos_entrenamiento,
                                               objetivo_entrenamiento,
                                               scoring='balanced_accuracy',
                                               cv=10, 
                                               n_jobs=-1,)
# Resultados_validación_cruzada
resultados_validación_cruzada['test_score'].mean()

0.917701390901188

In [9]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score

# Ajustar el modelo
clasificador_CART = DecisionTreeClassifier(max_depth=max_depth_mejor, min_samples_split=min_samples_split_mejor, random_state=42)
clasificador_CART.fit(atributos_entrenamiento, objetivo_entrenamiento)

# Calcular la Tasa de acierto del modelo
score = clasificador_CART.score(atributos_entrenamiento, objetivo_entrenamiento)

# Obtener las predicciones
predicciones = clasificador_CART.predict(atributos_entrenamiento)

# Contar los valores de la variable objetivo
values = pd.Series(objetivo_entrenamiento).value_counts()

# Calcular la matriz de confusión
m_confusion = confusion_matrix(objetivo_entrenamiento, predicciones)

# Calcular la sensibilidad
predicciones_prueba = clasificador_CART.predict(atributos_prueba)
recallscore = recall_score(objetivo_prueba, predicciones_prueba)

print(f'Variabes predictoras: \n{atributos_entrenamiento.columns}')
print()
print(f'Tasa de acierto: {score}')
print()
print(f'Valores: {values}')
print()
print(f'Matriz de confusión: \n{m_confusion}')
print()
print(f'Sensibilidad del conjunto de pruebas: {recallscore}')


Variabes predictoras: 
Index(['mean radius', 'mean texture', 'mean perimeter', 'mean area',
       'mean smoothness', 'mean compactness', 'mean concavity',
       'mean concave points', 'mean symmetry', 'mean fractal dimension',
       'radius error', 'texture error', 'perimeter error', 'area error',
       'smoothness error', 'compactness error', 'concavity error',
       'concave points error', 'symmetry error', 'fractal dimension error',
       'worst radius', 'worst texture', 'worst perimeter', 'worst area',
       'worst smoothness', 'worst compactness', 'worst concavity',
       'worst concave points', 'worst symmetry', 'worst fractal dimension'],
      dtype='object')

Tasa de acierto: 0.9868131868131869

Valores: diagnosis
1    285
0    170
Name: count, dtype: int64

Matriz de confusión: 
[[166   4]
 [  2 283]]

Sensibilidad del conjunto de pruebas: 1.0


### KNN

In [10]:
from sklearn.compose import ColumnTransformer

normalizador = ColumnTransformer([('normalizador',
                                   MinMaxScaler(feature_range=(0, 1)),
                                   atributos_continuos)])

In [7]:
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier

tubería_kNN = Pipeline([('preprocesador', normalizador),
                        ('kNN', KNeighborsClassifier())])
rejilla_de_parámetros = {
    # Número de vecinos impar (tarea de clasificación binaria)
    'kNN__n_neighbors': range(1, 10, 2),
    # Considerar las distancias Manhattan y euclídea
    'kNN__metric': ['manhattan', 'euclidean']
}


búsqueda_en_rejilla = GridSearchCV(tubería_kNN,
                                   rejilla_de_parámetros,
                                   scoring='balanced_accuracy',
                                   cv=10, 
                                   n_jobs=-1)
búsqueda_en_rejilla.fit(atributos_entrenamiento, objetivo_entrenamiento)

# Guardar el valor de max_depth en una variable
mejores_parametros = búsqueda_en_rejilla.best_params_

n_neighbors_mejor = mejores_parametros.get('kNN__n_neighbors')
metric_mejor = mejores_parametros.get('kNN__metric')
mejor_score = búsqueda_en_rejilla.best_score_

print("Mejor valor de n_neighbors:", n_neighbors_mejor)
print("Mejor valor de mmetric:", metric_mejor)
print("Mejor score:", mejor_score)



Mejor valor de n_neighbors: 5
Mejor valor de mmetric: manhattan
Mejor score: 0.9594718922051578


In [11]:
clasificador_kNN = KNeighborsClassifier(n_neighbors=n_neighbors_mejor, metric=metric_mejor)

clasificador_kNN.fit(atributos_entrenamiento, objetivo_entrenamiento)


# Calcular la Tasa de acierto del modelo
score = clasificador_kNN.score(atributos_entrenamiento, objetivo_entrenamiento)

# Obtener las predicciones
predicciones = clasificador_kNN.predict(atributos_entrenamiento)

# Contar los valores de la variable objetivo
values = pd.Series(objetivo_entrenamiento).value_counts()

# Calcular la matriz de confusión
m_confusion = confusion_matrix(objetivo_entrenamiento, predicciones)

# Calcular la sensibilidad
predicciones_prueba = clasificador_kNN.predict(atributos_prueba)
recallscore = recall_score(objetivo_prueba, predicciones_prueba)

print(f'Variabes predictoras: {atributos_entrenamiento.columns}')
print(f'Tasa de acierto: {score}')
print(f'Valores: {values}')
print(f'Matriz de confusión: \n{m_confusion}')
print()
print(f'Sensibilidad del conjunto de pruebas: {recallscore}')

Variabes predictoras: Index(['mean radius', 'mean texture', 'mean perimeter', 'mean area',
       'mean smoothness', 'mean compactness', 'mean concavity',
       'mean concave points', 'mean symmetry', 'mean fractal dimension',
       'radius error', 'texture error', 'perimeter error', 'area error',
       'smoothness error', 'compactness error', 'concavity error',
       'concave points error', 'symmetry error', 'fractal dimension error',
       'worst radius', 'worst texture', 'worst perimeter', 'worst area',
       'worst smoothness', 'worst compactness', 'worst concavity',
       'worst concave points', 'worst symmetry', 'worst fractal dimension'],
      dtype='object')
Tasa de acierto: 0.9494505494505494
Valores: diagnosis
1    285
0    170
Name: count, dtype: int64
Matriz de confusión: 
[[152  18]
 [  5 280]]

Sensibilidad del conjunto de pruebas: 0.9722222222222222


## Algoritmo de búsqueda hacia atrás

In [12]:
# Hemos definido la función de búsqueda secuencia hacia atrás en una clase python dentro de la carpeta "funciones"

import funciones.BusquedaSecuencialAtras as bsa

bsatras = bsa.backward_sequential_search(breast_cancer, 'Diagnosis', model, 10, 6) # Guardamos el DataFrame en la variable bsatras
bsatras

,variables,size,score
18,"[mean radius, mean texture, mean perimeter, me...",19,0.963348
19,"[mean radius, mean texture, mean perimeter, me...",20,0.962402
17,"[mean radius, mean texture, mean perimeter, me...",18,0.961943
8,"[mean radius, mean texture, mean fractal dimen...",9,0.961448
22,"[mean radius, mean texture, mean perimeter, me...",23,0.961440
4,"[mean texture, worst area, worst smoothness, w...",5,0.961432
21,"[mean radius, mean texture, mean perimeter, me...",22,0.960486
29,"[mean radius, mean texture, mean perimeter, me...",29,0.960486
7,"[mean texture, mean fractal dimension, area er...",8,0.960486
6,"[mean texture, mean fractal dimension, worst a...",7,0.960486


### Árbol de decisión de clasificación(DecisionTreeClassifier)


In [13]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

for k in range(0, len(bsatras)-1, 1):
    selected_variables = bsatras.iloc[k]['variables']  # Obtener las variables de la mejor iteración
    solucion = breast_cancer.loc[:, selected_variables]
    
    # Dividir los datos
    X = breast_cancer[selected_variables]
    objetivo= breast_cancer['Diagnosis']
    X_train, X_test, y_train, y_test = train_test_split(X, objetivo, test_size=.2,stratify=objetivo)

    búsqueda_en_rejilla = GridSearchCV(clasificador_CART,
                                   rejilla_de_hiperparámetros,
                                   scoring='balanced_accuracy',
                                   cv=10, 
                                   n_jobs=-1)  

    búsqueda_en_rejilla.fit(X_train, y_train)
    
    # Obtener los mejores parámetros de la búsqueda en rejilla
    
    mejores_parametros = búsqueda_en_rejilla.best_params_

    # Guardar el valor de max_depth en una variable
    max_depth_mejor = mejores_parametros.get('max_depth')
    min_samples_split_mejor = mejores_parametros.get('min_samples_split')
    mejor_score = búsqueda_en_rejilla.best_score_

    print("Mejor valor de max_depth:", max_depth_mejor)
    print("Mejor valor de min_samples_split:", min_samples_split_mejor)
    print("Mejor score:", mejor_score)

    # Entrenar el modelo
    clasificador_CART = DecisionTreeClassifier(
        max_depth=max_depth_mejor,  # Máxima profundidad del árbol
        min_samples_split=min_samples_split_mejor  # Mínimo número de ejemplos para poder particionar
    )

    resultados_validación_cruzada = cross_validate(clasificador_CART,
                                                X_train,
                                                y_train,
                                                scoring='balanced_accuracy',
                                                cv=10)

    # Ajustar el modelo
    clasificador_CART = DecisionTreeClassifier(max_depth=4)
    clasificador_CART.fit(X_train, y_train)


    # Calcular la Tasa de acierto del modelo
    score = clasificador_CART.score(X_train, y_train)

    # Obtener las predicciones
    predicciones = clasificador_CART.predict(X_train)

    # Contar los valores de la variable objetivo
    values = pd.Series(y_train).value_counts()

    # Calcular la matriz de confusión
    m_confusion = confusion_matrix(y_train, predicciones)

    # Calcular la sensibilidad
    predicciones_prueba = clasificador_CART.predict(X_test)
    recallscore = recall_score(y_test, predicciones_prueba)

    print(f'Variabes predictoras: \n{selected_variables}')
    print(f'Tasa de acierto: {score}')
    print(f'Valores: {values}')
    print(f'Matriz de confusión: \n{m_confusion}')
    print()
    print()
    print(f'Sensibilidad del conjunto de pruebas: {recallscore}')

Mejor valor de max_depth: 5
Mejor valor de min_samples_split: 5
Mejor score: 0.942368154158215
Variabes predictoras: 
['mean radius', 'mean texture', 'mean perimeter', 'mean area', 'mean concavity', 'mean fractal dimension', 'texture error', 'area error', 'concave points error', 'symmetry error', 'fractal dimension error', 'worst radius', 'worst texture', 'worst area', 'worst smoothness', 'worst compactness', 'worst concavity', 'worst concave points', 'worst symmetry']
Tasa de acierto: 0.9868131868131869
Valores: Diagnosis
1    285
0    170
Name: count, dtype: int64
Matriz de confusión: 
[[164   6]
 [  0 285]]


Sensibilidad del conjunto de pruebas: 0.9583333333333334
Mejor valor de max_depth: 5
Mejor valor de min_samples_split: 5
Mejor score: 0.9433243987250073
Variabes predictoras: 
['mean radius', 'mean texture', 'mean perimeter', 'mean area', 'mean concavity', 'mean fractal dimension', 'texture error', 'area error', 'concave points error', 'symmetry error', 'fractal dimension error

Mejor valor de max_depth: 5
Mejor valor de min_samples_split: 10
Mejor score: 0.9407671689365401
Variabes predictoras: 
['mean radius', 'mean texture', 'mean perimeter', 'mean area', 'mean fractal dimension', 'texture error', 'area error', 'concave points error', 'symmetry error', 'fractal dimension error', 'worst radius', 'worst area', 'worst smoothness', 'worst compactness', 'worst concavity', 'worst concave points', 'worst symmetry']
Tasa de acierto: 0.978021978021978
Valores: Diagnosis
1    285
0    170
Name: count, dtype: int64
Matriz de confusión: 
[[166   4]
 [  6 279]]


Sensibilidad del conjunto de pruebas: 0.9305555555555556
Mejor valor de max_depth: 3
Mejor valor de min_samples_split: 5
Mejor score: 0.9249855114459578
Variabes predictoras: 
['mean radius', 'mean texture', 'mean perimeter', 'mean area', 'mean smoothness', 'mean compactness', 'mean concavity', 'mean concave points', 'mean symmetry', 'mean fractal dimension', 'texture error', 'perimeter error', 'area error', 's

Mejor valor de max_depth: 4
Mejor valor de min_samples_split: 5
Mejor score: 0.9343161402492031
Variabes predictoras: 
['mean texture', 'worst area', 'worst smoothness']
Tasa de acierto: 0.978021978021978
Valores: Diagnosis
1    285
0    170
Name: count, dtype: int64
Matriz de confusión: 
[[164   6]
 [  4 281]]


Sensibilidad del conjunto de pruebas: 0.9722222222222222
Mejor valor de max_depth: 3
Mejor valor de min_samples_split: 5
Mejor score: 0.9394885540423067
Variabes predictoras: 
['worst area', 'worst smoothness']
Tasa de acierto: 0.9692307692307692
Valores: Diagnosis
1    285
0    170
Name: count, dtype: int64
Matriz de confusión: 
[[158  12]
 [  2 283]]


Sensibilidad del conjunto de pruebas: 0.9861111111111112


### KNN

In [14]:
from sklearn.naive_bayes import CategoricalNB


for k in range(0, len(bsatras)-1, 1):

    selected_variables = bsatras.iloc[k]['variables']  # Obtener las variables de la mejor iteración
    solucion = breast_cancer.loc[:, selected_variables]

    X = breast_cancer[selected_variables]
    X_train, X_test, y_train, y_test = train_test_split(X, objetivo, test_size=.2,stratify=objetivo)

    clasificador_kNN.fit(X_train, y_train)
    
    búsqueda_en_rejilla = GridSearchCV(tubería_kNN,
                                   rejilla_de_parámetros,
                                   scoring='balanced_accuracy',
                                   cv=10, 
                                   n_jobs=-1) 

    clasificador_kNN.fit(X_train, y_train)

    # Calcular la tasa de acierto del modelo
    score = clasificador_kNN.score(X_train, y_train)

    # Obtener las predicciones
    predicciones = clasificador_kNN.predict(X_train)

    # Contar los valores de la variable objetivo
    values = pd.Series(y_train).value_counts()

    # Calcular la matriz de confusión
    m_confusion = confusion_matrix(y_train, predicciones)

    # Calcular la sensibilidad
    predicciones_prueba = clasificador_kNN.predict(X_train)
    recallscore = recall_score(y_train, predicciones_prueba)

    print(f'Variabes predictoras: {selected_variables}')
    print(f'Tasa de acierto: {score}')
    print(f'Valores: {values}')
    print(f'Matriz de confusión: \n{m_confusion}')
    print()
    print(f'Sensibilidad del conjunto de pruebas: {recallscore}')

Variabes predictoras: ['mean radius', 'mean texture', 'mean perimeter', 'mean area', 'mean concavity', 'mean fractal dimension', 'texture error', 'area error', 'concave points error', 'symmetry error', 'fractal dimension error', 'worst radius', 'worst texture', 'worst area', 'worst smoothness', 'worst compactness', 'worst concavity', 'worst concave points', 'worst symmetry']
Tasa de acierto: 0.9736263736263736
Valores: Diagnosis
1    285
0    170
Name: count, dtype: int64
Matriz de confusión: 
[[161   9]
 [  3 282]]

Sensibilidad del conjunto de pruebas: 0.9894736842105263
Variabes predictoras: ['mean radius', 'mean texture', 'mean perimeter', 'mean area', 'mean concavity', 'mean fractal dimension', 'texture error', 'area error', 'concave points error', 'symmetry error', 'fractal dimension error', 'worst radius', 'worst texture', 'worst area', 'worst smoothness', 'worst compactness', 'worst concavity', 'worst concave points', 'worst symmetry', 'worst fractal dimension']
Tasa de acierto

Variabes predictoras: ['mean texture', 'mean fractal dimension', 'worst area', 'worst smoothness', 'worst concavity', 'worst symmetry']
Tasa de acierto: 0.9626373626373627
Valores: Diagnosis
1    285
0    170
Name: count, dtype: int64
Matriz de confusión: 
[[161   9]
 [  8 277]]

Sensibilidad del conjunto de pruebas: 0.9719298245614035
Variabes predictoras: ['mean texture', 'worst area', 'worst smoothness', 'worst concavity']
Tasa de acierto: 0.9758241758241758
Valores: Diagnosis
1    285
0    170
Name: count, dtype: int64
Matriz de confusión: 
[[162   8]
 [  3 282]]

Sensibilidad del conjunto de pruebas: 0.9894736842105263
Variabes predictoras: ['mean radius', 'mean texture', 'mean perimeter', 'mean area', 'mean smoothness', 'mean concavity', 'mean concave points', 'mean symmetry', 'mean fractal dimension', 'texture error', 'perimeter error', 'area error', 'compactness error', 'concavity error', 'concave points error', 'symmetry error', 'fractal dimension error', 'worst radius', 'wors

## Algoritmo de búsqueda hacia atrás mixta

In [15]:
# Hemos definido la función de búsqueda secuencia hacia atrás mixta en una clase python dentro de la carpeta "funciones"

import funciones.BusquedaSecuencialAtrasMixta as bsam

busq_atras_mixta = bsam.backward_sequential_mixed_search(breast_cancer, 'Diagnosis', model, 10, 6, 10) # Guardamos el DataFrame en la variable busq_atras_mixta
busq_atras_mixta

,variables,size,score
22,"[mean texture, concave points error, worst rad...",15,0.968974
20,"[mean texture, mean area, radius error, concav...",15,0.968966
21,"[mean texture, mean area, concave points error...",15,0.968958
11,"[mean texture, mean area, mean smoothness, mea...",19,0.968950
17,"[mean texture, mean area, mean symmetry, radiu...",18,0.968523
16,"[mean texture, mean area, mean symmetry, radiu...",18,0.968012
23,"[mean texture, concave points error, worst rad...",15,0.967119
14,"[mean texture, mean area, mean smoothness, mea...",18,0.966607
38,"[area error, smoothness error, worst symmetry,...",22,0.966591
19,"[mean texture, mean area, radius error, concav...",16,0.966172


### Árbol de decisión de clasificación(DecisionTreeClassifier)

In [18]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

for k in range(0, len(busq_atras_mixta)-1, 1):
    selected_variables = busq_atras_mixta.iloc[k]['variables']  # Obtener las variables de la mejor iteración
    solucion = breast_cancer.loc[:, selected_variables]
    
    model = DecisionTreeClassifier(max_depth=max_depth_mejor)

    # Dividir los datos
    X = breast_cancer[selected_variables]
    objetivo= breast_cancer['Diagnosis']
    X_train, X_test, y_train, y_test = train_test_split(X, objetivo, test_size=.2,stratify=objetivo)

    búsqueda_en_rejilla = GridSearchCV(clasificador_CART,
                                   rejilla_de_hiperparámetros,
                                   scoring='balanced_accuracy',
                                   cv=10, 
                                   n_jobs=-1)  

    búsqueda_en_rejilla.fit(X_train, y_train)
    
   # Obtener los mejores parámetros de la búsqueda en rejilla
    mejores_parametros = búsqueda_en_rejilla.best_params_

    # Guardar el valor de max_depth en una variable
    max_depth_mejor = mejores_parametros.get('max_depth')
    min_samples_split_mejor = mejores_parametros.get('min_samples_split')
    mejor_score = búsqueda_en_rejilla.best_score_

    print("Mejor valor de max_depth:", max_depth_mejor)
    print("Mejor valor de min_samples_split:", min_samples_split_mejor)
    print("Mejor score:", mejor_score)


    # Entrenar el modelo
    clasificador_CART = DecisionTreeClassifier(
        max_depth=max_depth_mejor,  # Máxima profundidad del árbol
        min_samples_split=min_samples_split_mejor  # Mínimo número de ejemplos para poder particionar
    )

    resultados_validación_cruzada = cross_validate(clasificador_CART,
                                                X_train,
                                                y_train,
                                                scoring='balanced_accuracy',
                                                cv=10)

    # Ajustar el modelo
    clasificador_CART = DecisionTreeClassifier(max_depth=max_depth_mejor)
    clasificador_CART.fit(X_train, y_train)

    # Calcular la Tasa de acierto del modelo
    score = clasificador_CART.score(X_train, y_train)

    # Obtener las predicciones
    predicciones = clasificador_CART.predict(X_train)

    # Contar los valores de la variable objetivo
    values = pd.Series(y_train).value_counts()

    # Calcular la matriz de confusión
    m_confusion = confusion_matrix(y_train, predicciones)

    # Calcular la sensibilidad
    predicciones_prueba = clasificador_CART.predict(X_test)
    recallscore = recall_score(y_test, predicciones_prueba)

    print(f'Variabes predictoras: \n{selected_variables}')
    print(f'Tasa de acierto: {score}')
    print(f'Valores: {values}')
    print(f'Matriz de confusión: \n{m_confusion}')
    print()  
    print(f'Sensibilidad del conjunto de pruebas: {recallscore}')

Mejor valor de max_depth: 6
Mejor valor de min_samples_split: 5
Mejor score: 0.9339321935670821
Variabes predictoras: 
['mean texture', 'concave points error', 'worst radius', 'worst texture', 'worst concavity', 'worst concave points', 'worst fractal dimension', 'area error', 'smoothness error', 'worst symmetry', 'mean smoothness', 'compactness error', 'mean fractal dimension', 'worst smoothness', 'radius error']
Tasa de acierto: 0.9912087912087912
Valores: Diagnosis
1    285
0    170
Name: count, dtype: int64
Matriz de confusión: 
[[168   2]
 [  2 283]]

Sensibilidad del conjunto de pruebas: 0.9166666666666666
Mejor valor de max_depth: 8
Mejor valor de min_samples_split: 5
Mejor score: 0.9384743552593451
Variabes predictoras: 
['mean texture', 'mean area', 'radius error', 'concave points error', 'worst radius', 'worst texture', 'worst concavity', 'worst concave points', 'worst fractal dimension', 'area error', 'smoothness error', 'worst symmetry', 'mean smoothness', 'compactness error

Mejor valor de max_depth: 5
Mejor valor de min_samples_split: 5
Mejor score: 0.9325304259634889
Variabes predictoras: 
['mean texture', 'mean area', 'mean symmetry', 'radius error', 'perimeter error', 'compactness error', 'concavity error', 'concave points error', 'symmetry error', 'worst radius', 'worst texture', 'worst concavity', 'worst concave points', 'worst fractal dimension', 'area error', 'smoothness error', 'worst symmetry', 'mean smoothness']
Tasa de acierto: 0.9912087912087912
Valores: Diagnosis
1    285
0    170
Name: count, dtype: int64
Matriz de confusión: 
[[166   4]
 [  0 285]]

Sensibilidad del conjunto de pruebas: 0.9583333333333334
Mejor valor de max_depth: 7
Mejor valor de min_samples_split: 5
Mejor score: 0.9410460736018547
Variabes predictoras: 
['mean texture', 'mean area', 'mean smoothness', 'mean symmetry', 'radius error', 'texture error', 'perimeter error', 'compactness error', 'concavity error', 'concave points error', 'symmetry error', 'worst radius', 'worst

Mejor valor de max_depth: 6
Mejor valor de min_samples_split: 15
Mejor score: 0.9477434077079107
Variabes predictoras: 
['mean texture', 'worst radius', 'worst concave points', 'area error', 'smoothness error', 'worst symmetry', 'mean smoothness', 'compactness error', 'mean fractal dimension', 'worst smoothness', 'radius error', 'worst concavity', 'fractal dimension error', 'worst texture']
Tasa de acierto: 0.9934065934065934
Valores: Diagnosis
1    285
0    170
Name: count, dtype: int64
Matriz de confusión: 
[[167   3]
 [  0 285]]

Sensibilidad del conjunto de pruebas: 0.9166666666666666
Mejor valor de max_depth: 4
Mejor valor de min_samples_split: 5
Mejor score: 0.9371341640104317
Variabes predictoras: 
['area error', 'smoothness error', 'worst symmetry', 'mean smoothness', 'compactness error', 'mean fractal dimension', 'worst smoothness', 'radius error', 'worst concavity', 'fractal dimension error', 'worst texture', 'mean area', 'concavity error']
Tasa de acierto: 0.9824175824175824

Mejor valor de max_depth: 10
Mejor valor de min_samples_split: 5
Mejor score: 0.930665024630542
Variabes predictoras: 
['area error', 'smoothness error', 'worst symmetry', 'mean smoothness', 'compactness error', 'mean fractal dimension', 'worst smoothness', 'radius error', 'worst concavity', 'fractal dimension error', 'worst texture', 'mean area', 'concavity error', 'mean compactness', 'texture error', 'worst fractal dimension', 'mean texture', 'symmetry error', 'worst area', 'perimeter error', 'mean concavity', 'mean perimeter', 'concave points error', 'worst radius', 'mean concave points', 'mean symmetry', 'worst compactness', 'worst concave points']
Tasa de acierto: 1.0
Valores: Diagnosis
1    285
0    170
Name: count, dtype: int64
Matriz de confusión: 
[[170   0]
 [  0 285]]

Sensibilidad del conjunto de pruebas: 0.9166666666666666
Mejor valor de max_depth: 8
Mejor valor de min_samples_split: 5
Mejor score: 0.9424913068675747
Variabes predictoras: 
['mean radius', 'mean texture', '

### KNN

In [21]:
from sklearn.naive_bayes import CategoricalNB


for k in range(0, len(busq_atras_mixta)-1, 1):

    selected_variables = busq_atras_mixta.iloc[k]['variables']  # Obtener las variables de la mejor iteración
    solucion = breast_cancer.loc[:, selected_variables]

    X = breast_cancer[selected_variables]
    X_train, X_test, y_train, y_test = train_test_split(X, objetivo, test_size=.2,stratify=objetivo)

    clasificador_kNN.fit(X_train, y_train)
    
    búsqueda_en_rejilla = GridSearchCV(tubería_kNN,
                                   rejilla_de_parámetros,
                                   scoring='balanced_accuracy',
                                   cv=10, 
                                   n_jobs=-1) 

    clasificador_kNN.fit(X_train, y_train)

    # Calcular la tasa de acierto del modelo
    score = clasificador_kNN.score(X_train, y_train)

    # Obtener las predicciones
    predicciones = clasificador_kNN.predict(X_train)

    # Contar los valores de la variable objetivo
    values = pd.Series(y_train).value_counts()

    # Calcular la matriz de confusión
    m_confusion = confusion_matrix(y_train, predicciones)

    # Calcular la sensibilidad
    predicciones_prueba = clasificador_kNN.predict(X_train)
    recallscore = recall_score(y_train, predicciones_prueba)

    print(f'Variabes predictoras: {selected_variables}')
    print(f'Tasa de acierto: {score}')
    print(f'Valores: {values}')
    print(f'Matriz de confusión: \n{m_confusion}')
    print()
    print(f'Sensibilidad del conjunto de pruebas: {recallscore}')

Variabes predictoras: ['mean texture', 'concave points error', 'worst radius', 'worst texture', 'worst concavity', 'worst concave points', 'worst fractal dimension', 'area error', 'smoothness error', 'worst symmetry', 'mean smoothness', 'compactness error', 'mean fractal dimension', 'worst smoothness', 'radius error']
Tasa de acierto: 0.9802197802197802
Valores: Diagnosis
1    285
0    170
Name: count, dtype: int64
Matriz de confusión: 
[[163   7]
 [  2 283]]

Sensibilidad del conjunto de pruebas: 0.9929824561403509
Variabes predictoras: ['mean texture', 'mean area', 'radius error', 'concave points error', 'worst radius', 'worst texture', 'worst concavity', 'worst concave points', 'worst fractal dimension', 'area error', 'smoothness error', 'worst symmetry', 'mean smoothness', 'compactness error', 'mean fractal dimension']
Tasa de acierto: 0.9736263736263736
Valores: Diagnosis
1    285
0    170
Name: count, dtype: int64
Matriz de confusión: 
[[159  11]
 [  1 284]]

Sensibilidad del con

Variabes predictoras: ['mean texture', 'mean area', 'mean smoothness', 'mean symmetry', 'radius error', 'texture error', 'perimeter error', 'compactness error', 'concavity error', 'concave points error', 'symmetry error', 'worst radius', 'worst texture', 'worst perimeter', 'worst concavity', 'worst concave points', 'worst fractal dimension', 'area error', 'smoothness error']
Tasa de acierto: 0.9692307692307692
Valores: Diagnosis
1    285
0    170
Name: count, dtype: int64
Matriz de confusión: 
[[157  13]
 [  1 284]]

Sensibilidad del conjunto de pruebas: 0.9964912280701754
Variabes predictoras: ['mean texture', 'worst radius', 'worst concave points', 'worst fractal dimension', 'area error', 'smoothness error', 'worst symmetry', 'mean smoothness', 'compactness error', 'mean fractal dimension', 'worst smoothness', 'radius error', 'worst concavity', 'fractal dimension error']
Tasa de acierto: 0.9736263736263736
Valores: Diagnosis
1    285
0    170
Name: count, dtype: int64
Matriz de confu

Variabes predictoras: ['area error', 'smoothness error', 'worst symmetry', 'mean smoothness', 'compactness error', 'mean fractal dimension', 'worst smoothness', 'radius error', 'worst concavity', 'fractal dimension error', 'worst texture', 'mean area', 'concavity error', 'mean compactness', 'texture error', 'worst fractal dimension', 'mean texture', 'symmetry error']
Tasa de acierto: 0.9604395604395605
Valores: Diagnosis
1    285
0    170
Name: count, dtype: int64
Matriz de confusión: 
[[155  15]
 [  3 282]]

Sensibilidad del conjunto de pruebas: 0.9894736842105263
Variabes predictoras: ['mean texture', 'mean perimeter', 'mean area', 'mean smoothness', 'mean compactness', 'mean concavity', 'mean concave points', 'mean symmetry', 'mean fractal dimension', 'radius error', 'texture error', 'perimeter error', 'area error', 'smoothness error', 'compactness error', 'concavity error', 'concave points error', 'symmetry error', 'worst radius', 'worst texture', 'worst perimeter', 'worst smoothne